## Non-variational fixed-depth Hamiltonian simulation

Here we time the FDHS example with non-variational KAK decomposition.

In [1]:
from functools import partial
from itertools import combinations, product
import numpy as np
import time
np.random.seed(2415)
from scipy.linalg import expm
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

import jax
jax.config.update("jax_enable_x64", True)

import pennylane as qml
from pennylane import X, Y, Z, I

from kak_tools import (
    map_simple_to_irrep,
    map_irrep_to_matrices,
    lie_closure_pauli_words,
    recursive_bdi,
    map_recursive_decomp_to_reducible,
    irrep_dot,
    make_signs,
)

In [2]:
# Helper functions
def make_so_2n_hamiltonian(n, coefficients="random"):
    couplings = [X(w) @ X(w+1) for w in range(n-1)] + [Y(w) @ Y(w+1) for w in range(n-1)]
    Zs = [Z(w) for w in range(n)]
    generators = couplings + Zs
    if coefficients == "random":
        alphas = np.random.normal(0.6, 1., size=n-1)
        betas = np.random.normal(0.3, 1.2, size=n-1)
        gammas = np.random.normal(0., 0.3, size=n)
    elif coefficients == "random TF":
        alphas = np.ones(n-1)
        betas = np.ones(n-1)
        gammas = np.random.normal(0., 0.3, size=n)
    elif coefficients == "uniform":
        alphas = np.ones(n-1)
        betas = np.ones(n-1)
        gammas = np.ones(n)
    
    coeffs = np.concatenate([alphas, betas, gammas])
    coeffs /= np.linalg.norm(coeffs) # Normalization
    H = qml.dot(coeffs, generators)
    generators = [next(iter(op.pauli_rep)) for op in generators]
    return H, generators, coeffs

def make_so_2n(n):
    algebra = [qml.pauli.PauliWord({w: P1, v: P2} | {i: "Z" for i in range(w+1, v)}) for w, v in combinations(range(n), r=2) for P1, P2 in product("XY", repeat=2)]
    algebra += [qml.pauli.PauliWord({w: "Z"}) for w in range(n)]
    return algebra
    
def make_so_2n_horizontal_mapping(n):
    mapping = {(i, i+n): qml.pauli.PauliWord({i: "Z"}) for i in range(n)}
    mapping |= {(i, i+n+1): qml.pauli.PauliWord({i: "X", i+1: "X"}) for i in range(n-1)}
    mapping |= {(i, i+n-1): qml.pauli.PauliWord({i-1: "Y", i: "Y"}) for i in range(1, n)}
    return mapping

def make_so_2n_full_mapping(n):
    mapping = {}
    for i in range(n-1):
        # upper left triangle
        mapping |= {(i, j): qml.pauli.PauliWord({i:"X", j:"Y"} | {w: "Z" for w in range(i+1, j)}) for j in range(i+1, n)}
        # lower right triangle
        mapping |= {(n+i, n+j): qml.pauli.PauliWord({i:"Y", j:"X"} | {w: "Z" for w in range(i+1, j)}) for j in range(i+1, n)}
        # upper right triangle of off-diagonal
        mapping |= {(i, n+j): qml.pauli.PauliWord({i:"X", j:"X"} | {w: "Z" for w in range(i+1, j)}) for j in range(i+1, n)}
        # lower left triangle of off-diagonal
        mapping |= {(j, n+i): qml.pauli.PauliWord({i:"Y", j:"Y"} | {w: "Z" for w in range(i+1, j)}) for j in range(i+1, n)}
    # diagonal of off-diagonal
    mapping |= {(i, n+i): qml.pauli.PauliWord({i:"Z"}) for i in range(n)}
    return mapping

In [3]:
# Config

coefficients = "random"
use_predefined_algebra = True
use_predefined_horizontal_mapping = True
use_hardcoded_mapping = True
ns = np.arange(2, 20, dtype=int) ** 2
ns = [6]

In [4]:
# times_mapping = []
# times_decomposing = []
# times_mapping_back = []
# times_rest = []
# times_total = []

for n in ns:
    n = int(n)
    H, generators, coeffs = make_so_2n_hamiltonian(n, coefficients)
    n_so = 2 * n
    so_dim = (n_so**2-n_so) // 2
    
    # start = time.process_time()

    if use_predefined_algebra:
        algebra = make_so_2n(n)
    else:
        algebra = lie_closure_pauli_words(generators, verbose=False)
    # start = time.process_time()
    assert len(algebra) == so_dim, f"{len(algebra)}, {so_dim}"

    # start_mapping = time.process_time()

    if not use_hardcoded_mapping:
        if use_predefined_horizontal_mapping:
            horizontal_ops = make_so_2n_horizontal_mapping(n)
        else:
            horizontal_ops = generators
        # print(f"Mapping simple to irrep")
        mapping, signs = map_simple_to_irrep(algebra, horizontal_ops=horizontal_ops, n=n_so, invol_type="BDI", invol_kwargs={"p": n_so//2, "q": (n_so+1)//2})
        # print(f"Mapping irrep to matrices")
    else:
        mapping = make_so_2n_full_mapping(n)
        signs = make_signs(mapping, n_so, "BDI")

    # end_mapping = time.process_time()

    epsilon = 0.01
    # print(f"Computing H and time evolution")
    H_irrep = irrep_dot(coeffs, generators, mapping, signs, n=n_so, invol_type="BDI")
    U = expm(epsilon * H_irrep)

    # print(f"Computing recursive decomposition")
    # start_decomposing = time.process_time()
    recursive_decomp = recursive_bdi(U, n_so)
    # end_decomposing = time.process_time()

    # print(f"Mapping recursive decomposition to Paulis")
    # start_mapping_back = time.process_time()
    pauli_decomp = map_recursive_decomp_to_reducible(recursive_decomp, mapping, signs, invol_type="BDI", time=epsilon, tol=None, assertions=False)
    # end_mapping_back = time.process_time()
    paulirot_decomp = [(coeff, qml.pauli.pauli_word_to_string(pw), pw.wires, _type) for pw, coeff, _type in pauli_decomp]

    # end = time.process_time()
    
    # times_mapping.append(end_mapping - start_mapping)
    # times_decomposing.append(end_decomposing - start_decomposing)
    # times_mapping_back.append(end_mapping_back - start_mapping_back)
    # times_total.append(end - start)
    # times_rest.append(times_total[-1] - times_mapping[-1] - times_mapping_back[-1] - times_decomposing[-1])
    
    # print(f"Decomposed exp(iHt) on {n} qubits into {len(paulirot_decomp)} Pauli rotations (DLA dimension: {so_dim}, Duration: {times_total[-1]:.3}s).")

In [5]:
paulirot_decomp = [(coeff, qml.pauli.pauli_word_to_string(pw), pw.wires, _type) for pw, coeff, _type in pauli_decomp]


In [6]:
def kak_time_evolution(time):
    # invert order because circuits and matrix products are written in opposite order
    for coeff, pauli_str, wires, _type in paulirot_decomp[::-1]:
        # Rescale the coefficient by the evolution time if the Pauli term is in the
        # central Cartan subalgebra
        if _type == "a0":
            coeff = coeff * time
        # Multiply by (-2) to undo the conventional builtin prefactor of -1/2 for PauliRot
        qml.PauliRot(-2 * coeff, pauli_word=pauli_str, wires=wires)

N = qml.dot([(1 - r) / 2 for r in range(1, n+1)], [Z(w) for w in range(n)]) + sum([(r-1)/2 for r in range(1, n+1)])
N_sq = qml.simplify(N @ N)

if n < 28:
    dev_light = qml.device("lightning.qubit", wires=n) # Fast simulator
    dev_def = qml.device("default.qubit", wires=n) # Slower but supports large dense matrices

from pennylane.devices.default_qubit import stopping_condition as sc

def stop(obj):
    return sc(obj) and len(obj.wires) <= 2

@partial(qml.transforms.decompose, gate_set=stop)
@qml.qnode(dev_light, grad_on_execution=False)
def kak_circuit(time):
    qml.X(0)
    kak_time_evolution(-time)
    return qml.expval(N_sq)

In [7]:
print(qml.draw(kak_circuit)(0.6))

0: ──X──────────RX(1.57)────╭X──RZ(-3.61)─╭X──RX(-1.57)─╭RYX(-3.14)──RX(1.57)───────────────────
1: ─╭RYX(3.03)───────────╭●─│─────────────│──╭●─────────╰RYX(-3.14)──RX(1.57)───────────────────
2: ─╰RYX(3.03)──H────────╰X─╰●────────────╰●─╰X──────────H───────────RX(1.57)────╭X──RZ(1.53)─╭X
3: ──RX(1.57)───────────────╭X──RZ(3.16)──╭X──RX(-1.57)─╭RYX(1.72)──╭X────────╭●─│────────────│─
4: ─╭RYX(1.54)───────────╭●─│─────────────│──╭●─────────╰RYX(1.72)──╰●────────│──│────────────│─
5: ─╰RYX(1.54)──H────────╰X─╰●────────────╰●─╰X──────────H───────────H────────╰X─╰●───────────╰●

───────────────────────────────────────────────────────────────────╭X──RZ(-3.14)─╭X──RX(-1.57)
──────────────────────╭X──RZ(0.71)─╭X──RX(-1.57)────╭X──────────╭●─│─────────────│──╭●────────
───RX(-1.57)────╭X─╭●─│────────────│──╭●─────────╭X─╰●──────────│──│─────────────│──│─────────
──╭●─────────╭X─╰●─│──│────────────│──│──────────╰●──H──────────╰X─╰●────────────╰●─╰X────────
──│──────────╰●──H─╰X─╰●───────────╰●